<a target="_blank" href="https://colab.research.google.com/drive/1sfOOyG7v2hAh2_nE_sMDT19Wp4lQYdf7?usp=sharing">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

## Introduction
Maxim AI provides comprehensive agent monitoring, evaluation, and observability for your Fireworks AI based applications. With Maxim’s one-line integration, you can easily trace and analyse agent interactions, performance metrics, and more.

## Install Required Dependencies

```bash
pip install fireworks-ai maxim-py
```

## Set the Environment Variables

In [ ]:
import os

MAXIM_API_KEY=os.getenv("MAXIM_API_KEY")
MAXIM_LOG_REPO_ID=os.getenv("MAXIM_REPO_ID")
FIREWORKS_API_KEY=os.getenv("FIREWORKS_API_KEY")

## Configure Maxim Logger

In [ ]:
from maxim import Config, Maxim
from maxim.logger import LoggerConfig

maxim = Maxim(Config(api_key=MAXIM_API_KEY))
logger = maxim.logger(LoggerConfig(id=MAXIM_LOG_REPO_ID))

/tmp/ipython-input-2152552063.py:5: DeprecationWarning: This class will be removed in a future version. Use {} which is TypedDict.
  maxim = Maxim(Config(api_key=os.getenv("MAXIM_API_KEY")))


[MaximSDK] Initializing Maxim AI(v3.10.0)


/tmp/ipython-input-2152552063.py:6: DeprecationWarning: This class will be removed in a future version. Use LoggerConfigDict instead.
  logger = maxim.logger(LoggerConfig(id=os.getenv("MAXIM_LOG_REPO_ID")))


In [6]:
from fireworks import LLM
from maxim.logger.fireworks import instrument_fireworks

instrument_fireworks(logger)

llm = LLM(
  model="qwen3-235b-a22b",
  deployment_type="serverless"
)

## Simple Inference

In [ ]:
response = llm.chat.completions.create(
  messages=[{
    "role": "user",
    "content": "Say this is a test",
  }],
)

response.choices[0].message.content

## Streaming Responses

In [ ]:
llm = LLM(
  model="qwen3-235b-a22b",
  deployment_type="serverless"
)
response_generator = llm.chat.completions.create(
  messages=[{
    "role": "user",
    "content": "Say this is a test",
  }],
  stream=True,
)
for chunk in response_generator:
    if chunk.choices[0].delta.content:
        print(chunk.choices[0].delta.content, end="")

## Tool Calls

In [ ]:

import json

llm = LLM(
    model="llama-v3p1-405b-instruct",
    deployment_type="serverless"
)

# Define the function tool for getting city population
tools = [
    {
        "type": "function",
        "function": {
            # The name of the function
            "name": "get_city_population",
            # A detailed description of what the function does
            "description": "Retrieve the current population data for a specified city.",
            # Define the JSON schema for the function parameters
            "parameters": {
                # Always declare a top-level object for parameters
                "type": "object",
                # Properties define the arguments for the function
                "properties": {
                    "city_name": {
                        # JSON Schema type
                        "type": "string",
                        # A detailed description of the property
                        "description": "The name of the city for which population data is needed, e.g., 'San Francisco'."
                    },
                },
                # Specify which properties are required
                "required": ["city_name"],
            },
        },
    }
]

# Define a comprehensive system prompt
prompt = f"""
You have access to the following function:

Function Name: '{tools[0]["function"]["name"]}'
Purpose: '{tools[0]["function"]["description"]}'
Parameters Schema: {json.dumps(tools[0]["function"]["parameters"], indent=4)}

Instructions for Using Functions:
1. Use the function '{tools[0]["function"]["name"]}' to retrieve population data when required.
2. If a function call is necessary, reply ONLY in the following format:
   <function={tools[0]["function"]["name"]}>{{"city_name": "example_city"}}</function>
3. Adhere strictly to the parameters schema. Ensure all required fields are provided.
4. Use the function only when you cannot directly answer using general knowledge.
5. If no function is necessary, respond to the query directly without mentioning the function.

Examples:
- For a query like "What is the population of Toronto?" respond with:
  <function=get_city_population>{{"city_name": "Toronto"}}</function>
- For "What is the population of the Earth?" respond with general knowledge and do NOT use the function.
"""

# Initial message context
messages = [
    {"role": "system", "content": prompt},
    {"role": "user", "content": "What is the population of San Francisco?"}
]

# Call the model
chat_completion = llm.chat.completions.create(
    messages=messages,
    tools=tools,
    temperature=0.1
)

# Print the model's response
print(chat_completion.choices[0].message.model_dump_json(indent=4))

## Using JSON Mode

```bash
pip install pydantic
```

In [14]:
from pydantic import BaseModel, Field

### Define the Result Schema

In [15]:
class Result(BaseModel):
    winner: str

### Define Output Schema

In [16]:
chat_completion = llm.chat.completions.create(
    response_format={
        "type": "json_schema",
        "json_schema": {
            "name": "Result",
            "schema": Result.model_json_schema()
        }
    },
    messages=[
        {
            "role": "user",
            "content": "Who won the US presidential election in 2012? Reply just in one JSON.",
        },
    ],
)

In [ ]:
print(repr(chat_completion.choices[0].message.content))